# Stereo Calibration

In [14]:
import cv2
import numpy as np
import glob

In [15]:
images = glob.glob("./imgs/original/calibration/left/*.jpg")
images.sort()
i = 0
for img_name in images:
    img = cv2.imread(img_name)
    img = cv2.resize(img, (int(img.shape[1]/4), int(img.shape[0]/4)))
    cv2.imwrite("./imgs/scaled/calibration/left/" + str(i + 1) + ".jpg", img)
    i += 1

images = glob.glob("./imgs/original/calibration/right/*.jpg")
images.sort()
i = 0
for img_name in images:
    img = cv2.imread(img_name)
    img = cv2.resize(img, (int(img.shape[1]/4), int(img.shape[0]/4)))
    cv2.imwrite("./imgs/scaled/calibration/right/" + str(i + 1) + ".jpg", img)
    i += 1

In [43]:
# 设置寻找亚像素角点的参数，采用的停止准则是最大循环次数30和最大误差容限0.001
criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 30, 0.001)

# 获取标定板角点的位置
objp = np.zeros((8 * 8, 3), np.float32)
objp[:, :2] = np.mgrid[0:8, 0:8].T.reshape(-1, 2)  # 将世界坐标系建在标定板上，所有点的Z坐标全部为0，所以只需要赋值x和y

obj_points = []  # 存储3D点
left_img_points = []  # 存储左图2D点
right_img_points = [] # 存储右图2D点

# 获取对应文件夹下的所有图片，进行标定工作
left_images = glob.glob("./imgs/scaled/calibration/left/*.jpg")
right_images = glob.glob("./imgs/scaled/calibration/right/*.jpg")
# 需要对图片进行排序，不然之后的绘制过程可能会因为乱序而没有效果
left_images.sort()
right_images.sort()

assert len(left_images) == len(right_images)

images_pair = zip(left_images, right_images)

for l_img, r_img in images_pair:
    left_img = cv2.imread(l_img)
    left_gray = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    l_size = left_gray.shape[::-1]
    left_ret, left_corners = cv2.findChessboardCorners(left_gray, (8, 8), None)

    right_img = cv2.imread(r_img)
    right_gray = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)
    r_size = right_gray.shape[::-1]
    right_ret, right_corners = cv2.findChessboardCorners(right_gray, (8, 8), None)

    if left_ret and right_ret:
        obj_points.append(objp)
        # Right points
        right_corners2 = cv2.cornerSubPix(right_gray, right_corners, (5, 5), (-1, -1), criteria)
        right_img_points.append(right_corners2)

        # Left points
        left_corners2 = cv2.cornerSubPix(left_gray, left_corners, (5, 5), (-1, -1), criteria)
        left_img_points.append(left_corners2)
    else:
        print("Couldn't find chessboard on " + l_img + " and " + r_img)
        break

l_ret, l_mtx, l_dist, _, _ = cv2.calibrateCamera(obj_points, left_img_points, l_size, None, None)
r_ret, r_mtx, r_dist, _, _ = cv2.calibrateCamera(obj_points, right_img_points, r_size, None, None)

i = 0
pairs = zip(left_images, right_images)

for l_img, r_img in pairs:
    l_image = cv2.imread(l_img)
    h, w = l_image.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(r_mtx, r_dist, (w, h), 1, (w, h))
    l_dst = cv2.undistort(l_image, r_mtx, r_dist, None, newcameramtx)
    cv2.imwrite("./imgs/scaled/calibration_result/left/" + str(i + 1) + ".jpg", l_dst)
    r_image = cv2.imread(r_img)
    h, w = r_image.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(r_mtx, r_dist, (w, h), 1, (w, h))
    r_dst = cv2.undistort(r_image, r_mtx, r_dist, None, newcameramtx)
    cv2.imwrite("./imgs/scaled/calibration_result/right/" + str(i + 1) + ".jpg", r_dst)

    i = i + 1


In [55]:
# Stereo Calibration

flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC
# flags |= cv2.CALIB_SAME_FOCAL_LENGTH
flags |= cv2.CALIB_FIX_FOCAL_LENGTH
flags |= cv2.CALIB_FIX_ASPECT_RATIO
flags |= cv2.CALIB_FIX_K1
flags |= cv2.CALIB_FIX_K2
flags |= cv2.CALIB_FIX_K3
flags |= cv2.CALIB_FIX_K4
flags |= cv2.CALIB_FIX_K5

stereo_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)


ret, Camera1Mat, Dist1, Camera2Mat, Dist2, R, T, E, F = cv2.stereoCalibrate(obj_points, left_img_points, right_img_points,
                                                                            l_mtx, l_dist, r_mtx, r_dist, imageSize=l_size,
                                                                            criteria=stereo_criteria, flags = flags)


In [56]:
print(Camera1Mat)
print(Camera2Mat)

[[435.73720578   0.         345.51108921]
 [  0.         436.80091531 238.25453162]
 [  0.           0.           1.        ]]
[[568.06867976   0.         374.53265488]
 [  0.         572.4684764  242.78889177]
 [  0.           0.           1.        ]]


In [57]:
# Show a result
img_L = cv2.imread("./imgs/scaled/calibration_result/left/1.jpg")
img_R = cv2.imread("./imgs/scaled/calibration_result/right/1.jpg")

img_size = img_L.shape[:2][::-1]

R1, R2, P1, P2, Q, roi_left, roi_right = cv2.stereoRectify(Camera1Mat, Dist1, Camera2Mat, Dist2, img_size, R, T, flags = 1)
left_map1, left_map2 = cv2.initUndistortRectifyMap(Camera1Mat, Dist1, R1, P1, img_size, cv2.CV_16SC2)
right_map1, right_map2 = cv2.initUndistortRectifyMap(Camera2Mat, Dist2, R2, P2, img_size, cv2.CV_16SC2)


result_l = cv2.remap(img_L, left_map1, left_map2, cv2.INTER_LINEAR)
result_r = cv2.remap(img_R, left_map1, left_map2, cv2.INTER_LINEAR)

result = np.concatenate((result_l, result_r), axis=1)
# draw_lines(result)
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("result.jpg", result)

True

In [59]:
def draw_lines(img):
    img_size = img.shape
    ptsX = [i for i in range(0, img_size[0], img_size[0]//20)]
    ptsY = [0, img_size[1]]
    for i in range(len(ptsX)):
        cv2.line(img, (ptsY[0], ptsX[i]), (ptsY[1], ptsX[i]), (0, 0, 255), 1, 1)

In [61]:
result_l = cv2.imread("./imgs/scaled/calibration_result/left/1.jpg")
result_r = cv2.imread("./imgs/scaled/calibration_result/right/1.jpg")

result = np.concatenate((result_l, result_r), axis=1)
draw_lines(result)
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite("result.jpg", result)

True

In [48]:
ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
'''

def write_ply(fn, verts, colors):
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

In [67]:
# disparity range is tuned for 'aloe' image pair
window_size = 3
min_disp = 16
num_disp = 72-min_disp
stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
    numDisparities = num_disp,
    blockSize = 4,
    P1 = 8*3*window_size**2,
    P2 = 32*3*window_size**2,
    disp12MaxDiff = 50,
    uniquenessRatio = 10,
    speckleWindowSize = 100,
    speckleRange = 32
)

print('computing disparity...')
disp = stereo.compute(result_l, result_r).astype(np.float32) / 16.0

print('generating 3d point cloud...',)
# Q = np.float32([[1, 0, 0, 0],
#                 [0, 1, 0, 0], # turn points 180 deg around x-axis,
#                 [0, 0, 1, 0], # so that y-axis looks up
#                 [0, 0, 0, 1]])

f = 0.8*w                          # guess for focal length
Q = np.float32([[1, 0, 0, -0.5*w],
                [0,-1, 0,  0.5*h], # turn points 180 deg around x-axis,
                [0, 0, 0,     -f], # so that y-axis looks up
                [0, 0, 1,      0]])
points = cv2.reprojectImageTo3D(disp, Q)
colors = cv2.cvtColor(result_l, cv2.COLOR_BGR2RGB)
mask = disp > disp.min()
out_points = points[mask]
out_colors = colors[mask]
out_fn = 'out.ply'
write_ply(out_fn, out_points, out_colors)
print('%s saved' % out_fn)

cv2.imshow('left', result_l)
cv2.imshow('disparity', (disp-min_disp)/num_disp)
cv2.waitKey()
cv2.destroyAllWindows()

print('Done')

computing disparity...
generating 3d point cloud...
out.ply saved
Done
